# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!sudo apt-get install fdupes

# script

In [42]:
%%bash
cat << EOF > delete_file.py
#!/usr/bin/python

import sys
import os

delete_filepath = sys.argv[1];

open(delete_filepath, 'w').close()
os.remove(delete_filepath)
EOF

In [50]:
%%bash


CLEAN_ITERATIONS=0
DESPERATE_ITERATIONS=0

NUM_OF_COMPR=0
echo "$NUM_OF_COMPR" > tmp_compr_count

NUM_OF_DUPS=0
echo "$NUM_OF_DUPS" > tmp_dups_count

FIND_PROGRESS=0

find_inc () {
  FIND_PROGRESS=$(($FIND_PROGRESS + 1))
  echo "Running query $FIND_PROGRESS"
}

find_trash_files () {
  find drive/ -type f -iname '*nos*18*ans*' | grep -v "\.g" | grep -v "School" >> deleted_files.txt
  find_inc
  find drive/ -type f -iname '*.mov' | grep -v "IMG_0239" | grep -v "School" >> deleted_files.txt
  find_inc
  find drive/ -type f -iname '*Collaborating*With*Parents*' | grep -v "School" >> deleted_files.txt
  find_inc
  find 'drive/MyDrive/Drexel_Portfolio Artifacts' -type f -size +10M >> deleted_files.txt
  find_inc
  find 'drive/MyDrive/Drexel_Portfolio Artifacts' -type f -iname '*.jpg' >> deleted_files.txt
  find_inc
  find 'drive/MyDrive/Drexel_Portfolio Artifacts' -type f -iname '*.jpeg' >> deleted_files.txt
  find_inc
  find 'drive/MyDrive/Drexel_Portfolio Artifacts' -type f -iname '*.png' >> deleted_files.txt
  find_inc
  find 'drive/MyDrive/Drexel_Portfolio Artifacts' -type f -iname '*.mov' >> deleted_files.txt
  find_inc
  find 'drive/MyDrive/Drexel_Portfolio Artifacts' -type f -iname '*.mkv' >> deleted_files.txt
  find_inc
  find 'drive/MyDrive/Drexel_Portfolio Artifacts' -type f -iname '*.mp4' >> deleted_files.txt
  find_inc
  find drive/ -type f -iname '*.pdf' | grep -i 'textbook' >> deleted_files.txt
  find_inc
  find drive/ -type f -iname '434*mp*homework*.pdf' >> deleted_files.txt
  find_inc
  find drive/ -type f -iname '455BC*.pdf' >> deleted_files.txt
  find_inc
  find 'drive/MyDrive/Second Year' -type f -iname '*.pdf' >> deleted_files.txt
  find_inc
  find 'drive/MyDrive/First Year' -type f -iname '*.pdf' >> deleted_files.txt
  find_inc
  find 'drive/MyDrive/Podcast How to' -type f >> deleted_files.txt
  find_inc
  find 'drive/MyDrive/HR PSA 2014 B' -type f | grep -v 'GLiu AMansfield Sports and Racism' >> deleted_files.txt
  find_inc
  find drive/ -type f -iname '*Now Hiring 2018 GENERAL staff*.pdf' >> deleted_files.txt
  find_inc
  find drive/MyDrive/Cycles*/*/PDFs/* -type f | grep '.pdf' >> deleted_files.txt
  find_inc
  find drive/MyDrive/compositions -type f -iname '*.mscz' >> deleted_files.txt
  find_inc
  find drive/MyDrive/20*BECD*Biology/* -type f >> deleted_files.txt
  find_inc
}

perma_rm () {
  python delete_file.py "$1"
}

clean_trash_files_query () {
  cat deleted_files.txt | sort | uniq > deleted_files2.txt
  mv deleted_files2.txt deleted_files.txt
}

remove_trash_files () {
  while IFS= read -r line; do
    echo "Deleting $line"
    perma_rm "$line"
  done < deleted_files.txt
}

remove_duplicate_files () {
  fdupes -r -A -q -f drive/MyDrive 2> /dev/null | grep -v "^$" | while IFS= read -r line; do

    NUM_OF_DUPS=$(($(cat tmp_dups_count) + 1))
    echo "$NUM_OF_DUPS" > tmp_dups_count

    echo "Deleting $line"
    perma_rm "$line"
  done
}



clean_iteration () {

  CLEAN_ITERATIONS=$(($CLEAN_ITERATIONS + 1))

  echo "=== Querying for garbage ==="
  find_trash_files
  echo "=== Cleaning garbage query ==="
  clean_trash_files_query
  echo "=== Removing garbage files ==="
  remove_trash_files
  echo "=== Removing duplicate files ==="
  remove_duplicate_files

}

compress_then_delete () {
  echo "Archiving files"
  7z a -t7z -m0=lzma -mx=9 -mfb=64 -md=32m -ms=on archive-desperate.7z archive
  cp archive-desperate.7z drive/MyDrive/archive-desperate.7z

  echo "Deleting uncompressed files"
  rm -rf archive
}


compressed_files_inc () {
  NUM_OF_COMPR=$(($(cat tmp_compr_count) + 1))
  echo "$NUM_OF_COMPR" > tmp_compr_count
}

clean_desperate_iteration () {

  DESPERATE_ITERATIONS=$(($DESPERATE_ITERATIONS + 1))

  echo "=== Compressing files ==="

  find drive/ -type f -size +1M | head -n 50 | grep -v 'archive-desperate.7z' | grep -v 'Drive Manage.ipynb' | grep -v '\.g' | while IFS= read -r line; do
    compressed_files_inc

    COMPR_FILE_LOCATION=$( dirname "archive/$line" )

    mkdir -p "$COMPR_FILE_LOCATION"
    mv "$line" "archive/$line"

    echo "Preparing to archive $line"
  done

  compress_then_delete
}


clean_progress_report () {
  local NUM_OF_TRASH=$(cat deleted_files.txt | wc -l)
  local NOW=$(TZ="EST" date)

  local BYTES_USED_REPORT=$(du -d 0 drive/MyDrive | sed -r 's/\t([^\n]+)$//')

  echo "Clean Iterations: $CLEAN_ITERATIONS"
  echo "Desperate Iterations: $DESPERATE_ITERATIONS"
  echo ""
  echo "Removed $(cat tmp_dups_count) duplicate files."
  echo "Removed $NUM_OF_TRASH trash files."
  echo "Compressed $(cat tmp_compr_count) files to save space."
  echo ""
  echo "Completed $NOW"
  echo ""
  echo "Space used: $(($BYTES_USED_REPORT / 150000)) %"
}

while :
do

  BYTES_USED=$(du -d 0 drive/MyDrive | sed -r 's/\t([^\n]+)$//')

  if [[ $BYTES_USED -gt 11000000 ]]
    then
      echo "=== Running in desperate mode ==="
      clean_iteration
      clean_desperate_iteration
      clean_progress_report > drive/MyDrive/progress.txt

    else
      echo "=== Running in normal mode ==="
      clean_iteration
      clean_progress_report > drive/MyDrive/progress.txt
  fi

done



Process is interrupted.


# misc

In [44]:
!du -h -d 1 drive/MyDrive > temp && cat temp | grep "drive/MyDrive/gliu20@scarsdaleschools.org 2021-06-24 08:31" >> temp && cat temp | sort -h -r | uniq -u | grep "drive/MyDrive/"

11M	drive/MyDrive/Wall Street Journal Front Pages
7.5M	drive/MyDrive/application process
5.4M	drive/MyDrive/YPP Westchester
3.9M	drive/MyDrive/LRC Website
3.5M	drive/MyDrive/In-Class Essay #1 Resources
2.9M	drive/MyDrive/Summer
2.9M	drive/MyDrive/Science Olympiad Test Reserve
2.8M	drive/MyDrive/Frosh
1.9M	drive/MyDrive/More posters pics
1.9M	drive/MyDrive/Final Team PowerPoint Presentations
1.6M	drive/MyDrive/Unit III Resources
1.5M	drive/MyDrive/Hour of Code
1.5M	drive/MyDrive/Drexel_Portfolio Artifacts
939K	drive/MyDrive/Discipline Zoom Backgrounds
769K	drive/MyDrive/Butler 6 Service Learning Project
683K	drive/MyDrive/Unit One Test Resources
636K	drive/MyDrive/ece2t3
609K	drive/MyDrive/Free Materials: Planner and Template
396K	drive/MyDrive/Cycles (2)
320K	drive/MyDrive/Cycles
319K	drive/MyDrive/ece2t4
296K	drive/MyDrive/412 Algebra 2018-2019
254K	drive/MyDrive/Schematics
228K	drive/MyDrive/Cycles (1)
148K	drive/MyDrive/Release
124K	drive/MyDrive/scioly regional forms
124K	drive/MyD

In [45]:
!find * -type f | xargs -d '\n' du -sh | grep -v ".gdoc" | grep -v ".gdraw" | grep -v ".gsheet" | grep -v ".gslides" | sort -h | tail -n 100

^C


In [46]:
!du -d 0 drive/MyDrive | sed -r 's/\t([^\n]+)$//'

730374


In [47]:
!fdupes --help

Usage: fdupes [options] DIRECTORY...

 -r --recurse     	for every directory given follow subdirectories
                  	encountered within
 -R --recurse:    	for each directory given after this option follow
                  	subdirectories encountered within (note the ':' at
                  	the end of the option, manpage for more details)
 -s --symlinks    	follow symlinks
 -H --hardlinks   	normally, when two or more files point to the same
                  	disk area they are treated as non-duplicates; this
                  	option will change this behavior
 -n --noempty     	exclude zero-length files from consideration
 -A --nohidden    	exclude hidden files from consideration
 -f --omitfirst   	omit the first file in each set of matches
 -1 --sameline    	list each set of matches on a single line
 -S --size        	show size of duplicate files
 -m --summarize   	summarize dupe information
 -q --quiet       	hide progress indicator
 -d --delete      	prompt user for files